In [4]:
import tensorflow as tf
import numpy as np
import src

In [6]:
problem = src.SortingProblem()

In [7]:
network = src.PointerNetwork('cfg')

seqLen = network.maxTimeSteps
batchSize = network.batchSize

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [4]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for batch in range(2000):
        inputs = problem.makeBatch(batchSize, 1, 100, seqLen)
        trainingInputs, targets = problem.makeTargets(inputs)
        
        _, loss = sess.run([network.trainOp, network.loss], 
                           feed_dict={network.train: True,
                                      network.rawInputs: inputs,
                                    network.targets: targets,
                                    network.targetInputs: trainingInputs[:,:-1,:]})
        
        if batch % 100 == 0:
            print('Batch: ' + str(batch))
            newInputs = problem.makeBatch(batchSize, 150, 600, seqLen)
            output = sess.run(network.decoderOutputs, feed_dict={network.train: False,
                                                                 network.rawInputs: newInputs})
            print(newInputs[0, :, :].T[0])
            print(newInputs[0, :, :].T[0][np.array(output).T[0]])
            

Batch: 0
[532 152 592 586 536]
[586 592 536 532 152]
Batch: 100
[538 519 490 226 220]
[490 220 519 226 538]
Batch: 200
[164 577 564 348 416]
[164 416 577 564 348]
Batch: 300
[261 276 357 432 463]
[357 276 463 432 261]
Batch: 400
[538 256 283 334 404]
[538 256 404 334 283]
Batch: 500
[411 253 406 426 502]
[253 406 411 502 426]
Batch: 600
[561 490 309 582 544]
[561 544 582 309 490]
Batch: 700
[412 577 487 290 188]
[412 188 290 577 487]
Batch: 800
[372 528 428 516 258]
[428 528 516 372 258]
Batch: 900
[155 544 352 295 269]
[155 269 295 352 544]
Batch: 1000
[241 295 256 320 226]
[295 226 241 256 320]
Batch: 1100
[580 570 194 585 228]
[194 570 580 585 228]
Batch: 1200
[528 409 378 509 239]
[528 239 509 378 409]
Batch: 1300
[373 177 494 165 583]
[165 494 583 373 177]
Batch: 1400
[326 546 544 213 195]
[213 195 326 546 544]
Batch: 1500
[493 525 399 351 439]
[439 493 525 399 351]
Batch: 1600
[300 264 384 372 553]
[264 553 384 300 372]
Batch: 1700
[525 384 206 239 309]
[206 525 309 384 239]
Batc